# Introduction

This notebook showcases my implementation of the Google Maps API to create a DataFrame of local food truck data in my local city, West Lafayette!

You will need to install the libraries geopy, ipyleaflet, and googlemaps. As well as pandas, numpy, and requests. Additionally, for dynamic DataFrame creation, you will need a Google API key. You get $200 in free credits each month, which is more than enough for a few tests.

## Setup

In [ ]:
#Installs necessary libraries
!pip install geopy
!pip install ipyleaflet
!pip install googlemaps
!pip install ipywidgets
!jupyter labextension install @jupyter-widgets/jupyterlab-manager
!jupyter labextension install jupyter-leaflet
print("Ready to continue.")

In [ ]:
import pandas as pd
import numpy as np
import requests
import googlemaps
from ipywidgets import HTML
from ipyleaflet import Map, basemaps, Marker, AntPath, Popup
from geopy import distance
import json
import os
from ast import literal_eval


print('Setup done! Boiler up!')

To reproduce the results, please write an API key here:

In [ ]:
%env API_KEY = "INSERT_KEY"

In [ ]:
#Access API key from environment and creates a new Google maps client
api_key = os.getenv('API_KEY')
gmaps = googlemaps.Client(key=api_key)

In [ ]:
#Sets our query and location for our food truck search
query = "food trucks"
#West Lafyette coordinates
location = (40.425869, -86.908066)
radius = 2

In [ ]:
#Uses Google places API to create a dictionary of search results
food_truck_search = gmaps.places(query, location, radius)

In [ ]:
#Creates a dataframe from the search results
df = pd.DataFrame(food_truck_search['results'])
df = df.set_index('name')

In [ ]:
#Obtains place info from each of the listings collected
truck_id = df['place_id'].to_list()
all_trucks = list()
for place_id in truck_id:
    place_info = gmaps.place(place_id)
    all_trucks.append(place_info)

In [ ]:
#Converts time format to more easily readable
def convert_time(number_str):
    if len(number_str) == 4:
        hours = int(number_str[:2])
        minutes = int(number_str[2:])
        if 0 <= hours < 24 and 0 <= minutes < 60:
            if hours < 12:
                am_pm = "am"
                if hours == 0:
                    hours = 12
            else:
                am_pm = "pm"
                if hours > 12:
                    hours -= 12
            formatted_time = f"{hours:02d}:{minutes:02d} {am_pm}"
            return formatted_time
    return "Invalid input"

In [ ]:
hours_dict = {}
for place in all_trucks:
    opening_hours = place['result'].get('opening_hours')
    if not opening_hours:
        None
    elif opening_hours.keys():
        hours_dict[place['result'].get('name')] = (opening_hours.get('periods'))

final_hours_dict = {}

for truck in hours_dict:
    for detail in hours_dict[truck]:
        if detail['open'].get('day') == 5:
            open_time = convert_time(detail['open']['time'])
            close_time = convert_time(detail['close']['time'])
            final_hours_dict[truck] = '{} - {}'.format(open_time, close_time)
        elif not detail['open'].get('day') == 5:
            for i in range(0,6):
                if detail['open'].get('day') == i:
                    open_time = convert_time(detail['open']['time'])
                    close_time = convert_time(detail['close']['time'])
                    final_hours_dict[truck] = '{} - {}'.format(open_time, close_time)

open_hours = pd.Series(final_hours_dict, name='Open Hours').fillna("Unknown")

In [ ]:
#Generates a "weighted" review score. A food truck with more reviews will have a greater value placed on their
#overall ranking. 
def calc_weighted_review(rating, num_reviews, weight):
    weighted_num_reviews = num_reviews ** weight
    weighted_score = (rating * weighted_num_reviews) / (weighted_num_reviews + 1)
    return weighted_score.round(2)

In [ ]:
weighted_review = calc_weighted_review(df['rating'], df['user_ratings_total'], 0.75).reindex(df.index).rename('Weighted Score')

In [ ]:
formatted_address = df['formatted_address'].rename("Address")
review_score = df['rating'].rename("Google Rating")

In [ ]:
websites = pd.Series([place['result'].get('website') for place in all_trucks], name = "Website")
phone_num = pd.Series([place['result'].get('formatted_phone_number') for place in all_trucks], name = "Phone Number")
websites.index = phone_num.index = df.index

In [ ]:
food_truck_df = pd.concat([formatted_address, open_hours, review_score, weighted_review, websites, phone_num], axis=1)
food_truck_df = food_truck_df.reset_index().rename(columns={'index':'Name'}).fillna("Unknown")

In [ ]:
lat_clean = {}
long_clean = {}
for i in range(0, len(df)):
    geometry_dict = df.iloc[i]['geometry']
    lat_clean[i] = (geometry_dict['location']['lat'])
    long_clean[i] = (geometry_dict['location']['lng'])
clean_loc = pd.DataFrame([lat_clean, long_clean])
clean_loc = clean_loc.rename({0:'Latitude', 1:'Longitude'}).T

tuples = list(zip(clean_loc['Latitude'], clean_loc['Longitude']))
tuples = pd.Series(tuples).rename('location')

Export and display final DataFrame:

In [ ]:
final_df = pd.concat([food_truck_df, tuples], axis=1)
final_df.rename(columns={'location':"Coordinates"}, inplace=True)

In [ ]:
final_df.to_csv("food_truck_data.csv", index=False)
print("Exported!")

# Import CSV
If you don't want to use an API key, start running code from here!

In [ ]:
final_df = pd.read_csv("food_truck_data.csv")
location = (40.425869, -86.908066) #West Lafayette

# Visualization
In this section, I will use ipyleaflet to visualize our ideal food truck route!

In [ ]:
name_fix = final_df.reset_index()
all_coords = list()
for coordinate in final_df['Coordinates']:
    all_coords.append(literal_eval(coordinate))

coordinates_series = pd.Series(all_coords, name="Coordinates")

food_truck_locations = pd.concat([name_fix['Name'], coordinates_series, final_df['Address']], axis=1)


In [ ]:
food_map = Map(
    basemap=basemaps.OpenStreetMap.Mapnik,
    center=location,
    zoom=13
)

food_trucks = [[x,y,z] for x, y, z in zip(food_truck_locations['Name'], food_truck_locations['Coordinates'], food_truck_locations['Address'])]
for truck in food_trucks:
    location = tuple(truck[1])
    marker = Marker(location=location, title = truck[0], draggable=False)
    food_map.add_layer(marker)
    detail = HTML()
    string = "<b>" + truck[0] + "</b>\n"+ truck[2] 
    detail.value = string
    
    popup = Popup(
    location=truck[1],
    child=detail,
    close_button=False,
    auto_close=False,
    close_on_escape_key=False,
    min_width= 2000,
    max_width = 20000
    )
    
    marker.popup = popup
    
food_map

#Creates a food map! Click the markers to see the address and name!
#Note there WILL be an error that pops up. I looked into this and found this issue: 
#https://github.com/jupyter-widgets/ipyleaflet/issues/1120
#The map is still functional.

Now that we have created our map, let's find out the most ideal route to take! (As a side note, ipyleaflet seems to have an unresolved issue with popups on the map. Link to issue [here](https://github.com/jupyter-widgets/ipyleaflet/issues/1120). 

# Distance Calculation

In [ ]:
distance_calc = food_truck_locations.dropna()

In [ ]:
distance_table = [[0 for i in range(len(distance_calc))] for j in range(len(distance_calc))]
distance_list = food_truck_locations['Coordinates']

In [ ]:
for i in range(len(distance_calc)):
    for j in range(len(distance_calc)):
        distance_table[i][j] = distance.distance(distance_list[i], distance_list[j]).miles

distance_table = pd.DataFrame(distance_table)

In [ ]:
temp_final_df = final_df.copy().set_index('Name')
distance_table.index = temp_final_df.index
distance_table = distance_table.T

In [ ]:
distance_table.index = temp_final_df.index

In [ ]:
distance_table.replace(0, np.nan, inplace=True)
min_list = distance_table.idxmin()
#Finds the nearest food truck from each other food truck.

In [ ]:
styled_distance = distance_table.style.background_gradient(cmap='viridis', vmin=0, vmax=5.0).set_properties(**{'font-size': '11px'})
styled_distance
#Presents distance table in a heatmap format

In [ ]:
min_list = min_list.reset_index(drop=True)
to_and_from_coords = list()
for i in range(len(distance_table)):
    start_loc = distance_table.iloc[i]
    to_loc = distance_table.iloc[i].idxmin()
    if food_truck_locations[food_truck_locations['Name'] == to_loc]['Coordinates'].any():
        to_loc = food_truck_locations[food_truck_locations['Name'] == to_loc]['Coordinates'].iloc[0]
        start_loc = food_truck_locations[food_truck_locations['Name'] == start_loc.name]['Coordinates'].iloc[0]
        to_and_from_coords.append((start_loc, to_loc))
# Determines the shortest route from each food truck to the next.

In [ ]:
# Adds the paths to the map!
for coords in to_and_from_coords:
    path = AntPath(locations = [coords[0], coords[1]], color='#000000',
    pulse_color='#CEB888')
    food_map.add_layer(path)

# Final Data Display


In [ ]:
display(food_map)

In [ ]:
display(final_df)

## Distances are in miles.

In [ ]:
display(styled_distance)

# Analysis
Here is my guide to the ultimate weekend filled with food trucks in the West Lafayette area, with a splash of Indianapolis. 

## About my DataFrame
My DataFrame consistst of a dynamically generated list of food trucks obtained by a query to the Google Maps API. I obtained this data in a series of dictionaries with a Google Maps library in Python. I cleaned up this data and narrowed down essential columns to determine what was necessary for someone interested in local food trucks. Name, address, website, phone number, and Google Rating were scraped directly from Google's Place API. Open hours were calculated from parsing through any available hours listings and applying the first result found. (**Note:** This method does extrapolate data, but for ease of access, displaying the first available time generally gives an idea of a truck's opening hours.) I then calculated a "weighted" review. This weighted review considers the amounts of ratings a truck has been given, and appropriately scales the average rating accordingly. I came up with this idea after considering my personal level of trust in reviews. I would rather go to a restaurant with 100 reviews, but a 4 star rating, than a restaurant with 3 reviews, but a 5 star rating. This also gives us an idea of how truly popular and well-reviewed the restaurant is overall, since a weighted score that is closer to the average indicates a high number of reviews. Coordinate data is also collected from the Google Maps query, which I use for my map visualizations later.

I intialized my DataFrame dynamically, hopefully allowing for any coordinate to be placed in the search query and for a result to be properly created.   

## About my visualizations
I utilized the ipyleaflet as well as the Geopy libraries in order to create my map visualizations. By collecting the coordinate data from Google Maps, I calculated several measures of distance. I utilized Geopy's distance function, which uses geodesic math to find the shortest distance between two points, taking note of the Earth's curvature. One visualization I created from this data was a visual distance heatmap. This would allow hungry searchers to determine the closest adjacent food truck. Distances are color coded in a heatmap format to allow for fast interpretation.
My next visualization involves the ipyleaflet library. This library allows me to output an interactive map onto the Juypter notebook interface. I constructed a dynamic map that will place markers on each of the found food trucks. I then used my distance calculation to form paths between the nearest trucks. This will allow for the ideal travel path to be found! (The paths are colored in Purdue's colors!)

Like my DataFrame, these maps were dynamically generated from the Google Maps query. This will allow for any new data to appropriately appear as needed. 

# The foodie plan

## Day 1:
Let's begin in West Lafayette, right near Purdue! We can grab lunch around 11:00 AM at **The Guac Box** (1400 West State St. STE K), this restaurant has a very good weighted score, and if we wanted to check what's on the menu, we can visit their [website](http://theguacbox.com/), so get ready for some great **tacos**! Optionally, we can visit the Farmer's Fridge (1198 3rd Street, West Lafayette), but we don't have a lot of information, and it has a pretty low score, so I wouldn't make it a priority. 


We then travel out of Purdue's town to head to neighboring Lafayette. A 3.6 mile drive from The Guac Box brings us to our next high scoring food truck, **Wicked Food Shack** (2070 S 22nd St). We can visit any time after lunch, make sure to give them a call at (765) 409-7743 to see if they're open, but we can get a second round of **tacos** here! 


Keep some room for more tacos around  at the **Tacos Mexico Food Truck** (3109 South St), just 1.5 miles away, we have another solid food truck. I would check their [menu](http://www.tacosmexicolafayette.com/) and maybe give them a call before visiting, since we were not able to obtain their hours. 

But hold on to your stomach! We'll definitely want to hit **Tortas Food Truck** (311 Sagamore Pkwy N), easily the highest score we've seen yet, just in time for dinner around 5 PM. We'll have to drive a quick 0.3 miles.


We have some time to relax from all of the tacos we've had so far, let's head back to Purdue's campus and get some studying in (it's midterms season!) Luckily for us, we don't have to spend all night, for we have a final food truck to visit today! **Famous Frank's** (30 Pierce St) is a classic spot for late night meals past 11PM. Grab some hot dogs and sandwiches here, and hopefully spot some other students here. This food truck is a little elusive, with no website or phone number easily found. (However, if we decided to stay near Tortas Food Truck for a while, it would have been a 2.8 mile drive back.)

And with that, we've finished day 1 of our foodie trip! Lots of tacos today.

## Day 2:

Luckily for us, we have another great food truck waiting for us back at Purdue for lunch around 11AM once again. **Grilled Chicken and Rice** (1400 W State St Suite M) is right next to where we started yesterday! After that, we're gonna head to Indianapolis and spend a day there. Lots to do, and lots to see, and lots to eat! 


We'll make sure to visit **The Latin Flavor**, about 64 miles away. We'll be able to get a new Indy taco taste! After that we can visit the new **Big Jerry Indy Food Truck**. A low weighted score simply corresponds to less reviews, but we can change that! We can grab some fresh sandwiches for dinner before 7PM, after spending a fun day in Indy!

# Conclusion
That concludes my 2 day foodie plan, as well as my discussion on the implementation of my DataFrame. I hope you'll find a good food truck to eat at near you!